<a href="https://colab.research.google.com/github/asheta66/CNN/blob/main/Breast_Diagnosis_Model_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# !pip install tensorflow-addons

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from PIL import Image
import itertools

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define your data directory path
data_dir = '/content/drive/My Drive/data2'

# Define possible resolution options and corresponding folder names
resolutionOptions = {
    '32x32': (32, 32),
    '56x56': (56, 56),
    '128x128': (128, 128),
    '256x256': (256, 256),
    '512x512': (512, 512)
}

# Ask the user to select a resolution
print("")   # Prints an empty line
print("Select the resolution:")
for i, (res_name, res_size) in enumerate(resolutionOptions.items(), 1):
    print(f"{i}. {res_name}")
print("")   # Prints an empty line

selectionIndex = int(input("Enter the number corresponding to your choice: ")) - 1
if selectionIndex < 0 or selectionIndex >= len(resolutionOptions):
    raise ValueError("Invalid selection. Exiting.")

# Construct the data directory path and resolution based on the selected resolution
selectedResolutionName = list(resolutionOptions.keys())[selectionIndex]
selectedResolutionSize = resolutionOptions[selectedResolutionName]
data_dir = os.path.join('/content/drive/My Drive/data2', selectedResolutionName)

# Display the selected directory path and resolution
print("")   # Prints an empty line
print(f"Selected data directory: {data_dir}")
print("")   # Prints an empty line
print(f"Selected resolution: {selectedResolutionSize}")
print("")   # Prints an empty line

# Check if the data directory exists
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Directory '{data_dir}' does not exist.")

# Get the list of class names (subfolders)
classes = [cls for cls in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, cls))]

for cls in classes:
    cls_dir = os.path.join(data_dir, cls)
    num_images = len(os.listdir(cls_dir))
    print(f"Class '{cls}': {num_images} images")


# Function to build and compile the CNN model
def build_compile_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy', precision, recall, f1_score])
    return model

# Custom metrics
def precision(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

def plot_confusion_matrices(cm_train, cm_test, classes, train_title, test_title, filename, font_size=10):
    fig, ax = plt.subplots(1, 2, figsize=(6, 5))

    # Training confusion matrix
    ax[0].imshow(cm_train, interpolation='nearest', cmap=plt.cm.Blues)
    ax[0].set_title(train_title, fontsize=font_size)
    tick_marks = np.arange(len(classes))
    ax[0].set_xticks(tick_marks)
    ax[0].set_yticks(tick_marks)
    ax[0].set_xticklabels(classes, rotation=45, fontsize=font_size)
    ax[0].set_yticklabels(classes, fontsize=font_size)
    fmt = 'd'
    thresh = cm_train.max() / 2.
    for i, j in itertools.product(range(cm_train.shape[0]), range(cm_train.shape[1])):
        ax[0].text(j, i, format(cm_train[i, j], fmt),
                   horizontalalignment="center",
                   color="white" if cm_train[i, j] > thresh else "black",
                   fontsize=font_size)
    ax[0].set_ylabel('True label', fontsize=font_size)
    ax[0].set_xlabel('Predicted label', fontsize=font_size)
    ax[0].grid(False)

    # Testing confusion matrix
    ax[1].imshow(cm_test, interpolation='nearest', cmap=plt.cm.Blues)
    ax[1].set_title(test_title, fontsize=font_size)
    ax[1].set_xticks(tick_marks)
    ax[1].set_yticks(tick_marks)
    ax[1].set_xticklabels(classes, rotation=45, fontsize=font_size)
    ax[1].set_yticklabels(classes, rotation=45, fontsize=font_size)
    fmt = 'd'
    thresh = cm_test.max() / 2.
    for i, j in itertools.product(range(cm_test.shape[0]), range(cm_test.shape[1])):
        ax[1].text(j, i, format(cm_test[i, j], fmt),
                   horizontalalignment="center",
                   color="white" if cm_test[i, j] > thresh else "black",
                   fontsize=font_size)
    ax[1].set_ylabel('True label', fontsize=font_size)
    ax[1].set_xlabel('Predicted label', fontsize=font_size)
    ax[1].grid(False)

    plt.tight_layout()
    plt.savefig(filename)
    plt.show()

# Function to load a single image
def load_image(directory, cls, img_file):
    img_path = os.path.join(directory, cls, img_file)
    img = Image.open(img_path)
    return img

# Lists to store results
histories = []
confusion_matrices_train = []
confusion_matrices_test = []
classification_reports_train = []
classification_reports_test = []

# Prepare data
X, y = [], []
label_map = {cls: idx for idx, cls in enumerate(classes)}

for cls in classes:
    cls_dir = os.path.join(data_dir, cls)
    img_files = os.listdir(cls_dir)

    for img_file in img_files:
        img_path = os.path.join(cls_dir, img_file)
        img = load_img(img_path, target_size=selectedResolutionSize)
        img_array = img_to_array(img)
        X.append(img_array)
        y.append(label_map[cls])

X = np.array(X).astype('float32') / 255.0
y = np.array(y).astype('int')

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=23)

input_shape = (selectedResolutionSize[0], selectedResolutionSize[1], 3)
num_classes = len(classes)

# Build and compile model
model = build_compile_model(input_shape, num_classes)

# Train the model with validation split
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Predict and evaluate
y_train_pred = np.argmax(model.predict(X_train), axis=1)
y_test_pred = np.argmax(model.predict(X_test), axis=1)

cm_train = confusion_matrix(y_train, y_train_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

report_train = classification_report(y_train, y_train_pred, target_names=classes)
report_test = classification_report(y_test, y_test_pred, target_names=classes)

# Store results
histories.append(history)
confusion_matrices_train.append(cm_train)
confusion_matrices_test.append(cm_test)
classification_reports_train.append(report_train)
classification_reports_test.append(report_test)

# Save the CNN architecture as a PNG figure
plot_model(model, to_file=f'model_architecture_{selectedResolutionSize[0]}x{selectedResolutionSize[1]}.png', show_shapes=True, show_layer_names=True)

# Draw a sample image (one image from each class)
fig, axes = plt.subplots(1, len(classes), figsize=(20, 20))
for ax, cls in zip(axes, classes):
    img_file = random.choice(os.listdir(os.path.join(data_dir, cls)))
    img = load_image(data_dir, cls, img_file)
    ax.imshow(img)
    ax.set_title(cls)
    ax.axis('off')

# Save the figure
plt.savefig(f'sample_images_{selectedResolutionSize[0]}x{selectedResolutionSize[1]}.png')
plt.close()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Select the resolution:
1. 32x32
2. 56x56
3. 128x128
4. 256x256
5. 512x512

Enter the number corresponding to your choice: 5

Selected data directory: /content/drive/My Drive/data2/512x512

Selected resolution: (512, 512)

Class 'benign': 891 images
Class 'normal': 266 images
Class 'malignant': 421 images


**Display confusion matrix and convergence curves**

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

# Define a resolution string for filenames
resolution_str = f'{selectedResolutionSize[0]}x{selectedResolutionSize[1]}'

# Save the confusion matrices as PNG
plot_confusion_matrices(cm_train, cm_test, classes,
                        f'Training CM - {resolution_str}',
                        f'Testing CM - {resolution_str}',
                        f'confusion_matrices_{resolution_str}.png',
                        font_size=11)

# Save the CNN architecture as a PNG figure
plot_model(model, to_file=f'model_architecture_{resolution_str}.png', show_shapes=True, show_layer_names=True)

# Plot convergence curves (accuracy and loss in one figure as subplots) and save as PNG
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 5))

# Accuracy plot
ax1.plot(history.history['accuracy'], label='Train Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.set_title(f'Accuracy - {resolution_str}')
ax1.grid()

# Loss plot
ax2.plot(history.history['loss'], label='Train Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.set_title(f'Loss - {resolution_str}')
ax2.grid()

plt.tight_layout()
plt.savefig(f'convergence_curve_{resolution_str}.png')
plt.show()


**Display the performance criteria**

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize a list to store metrics for saving
metrics = []

# After training and predictions
train_predictions = np.argmax(model.predict(X_train), axis=1)
test_predictions = np.argmax(model.predict(X_test), axis=1)

# Compute metrics
accuracy = accuracy_score(y_test, test_predictions)
precision = precision_score(y_test, test_predictions, average='weighted')
recall = recall_score(y_test, test_predictions, average='weighted')
f1 = f1_score(y_test, test_predictions, average='weighted')

# Store metrics in the list
metrics.append({
    'Resolution': f'{selectedResolutionSize[0]}x{selectedResolutionSize[1]}',
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
})

# Create a DataFrame from the metrics list
metrics_df = pd.DataFrame(metrics)

display(metrics_df)

# Define filename with resolution
excel_filename = f'performance_metrics_{selectedResolutionSize[0]}x{selectedResolutionSize[1]}.xlsx'

# Save the DataFrame to an Excel file
metrics_df.to_excel(excel_filename, index=False, sheet_name='Performance Metrics')

print(f"Metrics saved to {excel_filename}")
